# Performance Test

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
size = 1_000_000

df = pd.DataFrame({
    'buy_signal': np.random.randint(0, 2, size=size).astype(np.bool_),
    'sell_signal': np.random.randint(0, 2, size=size).astype(np.bool_)
})
# Ensure exclusivity of signals in each row
df['sell_signal'] = df['sell_signal'] & ~df['buy_signal']

In [3]:
def is_position(buy, sell, in_position):
    if buy:
        return True
    elif sell:
        return False
    else:
        return in_position

## Pandas iterrows()

In [ ]:
%%timeit
positions = []
in_position = False

for index, row in df.iterrows():
    in_position = is_position(row['buy_signal'], row['sell_signal'], in_position)
    positions.append(in_position)
    
df['in_position'] = positions

## Pandas apply()

In [ ]:
%%timeit
in_position = False
positions = df.apply(lambda row: is_position(row['buy_signal'], row['sell_signal'], in_position), axis=1)

df['in_position'] = positions

## numpy and ffill()

In [ ]:
%%timeit
signals = np.where(df['buy_signal'] == True, 1, 
                   np.where(df['sell_signal'] == True, -1, np.nan))

ffill_signals = pd.Series(signals).ffill().fillna(-1).astype(int)
positions = (ffill_signals + 1) // 2

df['in_position'] = positions.astype(np.bool_)